In [1]:
import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import PassiveAggressiveClassifier
# from sklearn.metrics import accuracy_score
# from collections import defaultdict
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import spacy
from datetime import datetime
nlp = spacy.load("en_core_web_md")

GLOBAL VARIABLE DEFINITION:

In [2]:
global global_list
global_list = [ 
    [
        ['keyword1', 'keyword2', 'keyword3'],
        [
            ['January 1, 2022', 'Text 1'],
            ['January 3, 2022', 'Text 2'],
            ['December 18, 2021', 'Text 3']
        ]
    ],
    [
        ['keyword4', 'keyword5', 'keyword6'],
        [
            ['January 10, 2022', 'Text 4'],
            ['January 30, 2022', 'Text 5'],
            ['April 12, 20221', 'Text 6']
        ]
    ]
]

All functions combined:

In [3]:
def extract_keywords(text, num_keywords=5):
    # Tokenize the text
    tokenizer = RegexpTokenizer(r'\b[a-zA-Z]{3,}\b')  # Keep only words with 3 or more characters
    tokens = tokenizer.tokenize(text)
    # Remove stopwords and lemmatize the tokens
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    filtered_tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stop_words]
    # Calculate word frequency
    freq_dist = FreqDist(filtered_tokens)
    # Get the most frequent non-stopword lemmatized words
    top_keywords = freq_dist.most_common(num_keywords)
    return [word for word, _ in top_keywords]


def most_similar_chain(input_keywords):
    max_similarity = 0
    pos = -1
    temp = -1
    for chain in global_list:  # Iterate over each chain in global_list
        temp += 1
        keywords = chain[0]  # Keywords are in the first element of the chain
        chain_items = chain[1]  # Text-date pairs are in the second element of the chain
        if not chain_items:  # Skip empty chains
            continue
        # Calculate similarity with keywords
        keyword_similarity = sum(nlp(keyword).similarity(nlp(input_keyword)) for keyword in keywords for input_keyword in input_keywords)
        # Calculate similarity with text in each text-date pair
        text_similarity = sum(nlp(text).similarity(nlp(input_keyword)) for date, text in chain_items for input_keyword in input_keywords for text in [text_date_pair[1]])
        # Calculate overall similarity as the sum of keyword similarity and text similarity
        similarity = (keyword_similarity + text_similarity) / (len(input_keywords) + len(chain_items))
        if similarity > max_similarity:
            max_similarity = similarity
            pos = temp
    if max_similarity < 0.5:
        pos = -1
    return pos





def chain_item_sorting(list_of_lists):
    # Sort the list of lists based on the date in each inner list
    temp_list = list_of_lists[1]
    sorted_list = sorted(list_of_lists[1:], key=lambda x: datetime.strptime(x[0][0], "%B %d, %Y"))
    list_of_lists[1] = sorted_list
    return

# def sort_by_date(date_text_pairs):
#     # Convert the date strings to datetime objects
#     for pair in date_text_pairs:
#         pair[0] = datetime.strptime(pair[0], '%B %d, %Y')
#     # Sort the list based on the datetime objects
#     sorted_pairs = sorted(date_text_pairs, key=lambda x: x[0])
#     # Convert the datetime objects back to strings
#     for pair in sorted_pairs:
#         pair[0] = datetime.strftime(pair[0], '%B %d, %Y')
#     return sorted_pairs



def add_to_global_var(text_input, date_added):
    kw_list = extract_keywords(text_input)
    pos = most_similar_chain(kw_list)
    if pos == -1:
        global_list.append([kw_list, [[date_added, text_input]]])
        chain_item_sorting(global_list[-1])
    else:
        global_list[pos].append([date_added, text_input])
        chain_item_sorting(global_list[pos])


def print_global_list():
    for chain in global_list:
        print("Keywords:", chain[0])
        print("Text-Date Pairs:")
        for pair in chain[1]:
            print("  Text:", pair)
            print()
        print()



In [4]:
# # Sample fake news text
# fake_news_text = """
# Breaking: Scientists confirm aliens exist and have made contact with Earth!
# According to reliable sources, extraterrestrial beings have been communicating with top government officials for years. The evidence suggests that they have advanced technology far beyond our understanding.
# """

# # Extract keywords
# keywords = extract_keywords(fake_news_text)
# print("Keywords for fake news detection:", keywords)

Function to find the most similar chain with input of list of keywords:

In [5]:
# Load the spaCy model


# def most_similar_chain(input_keywords):
#     max_similarity = 0
#     most_similar_chain = None

#     for item in global_list:  # Skip the first item which contains only keywords
#         keywords = item[0]  # Keywords are in the first element of each item
#         similarity = sum(nlp(keyword).similarity(nlp(input_keyword)) for keyword in keywords for input_keyword in input_keywords)
#         if similarity > max_similarity:
#             max_similarity = similarity
#             most_similar_chain = item
#     max_similarity = max_similarity/len(input_keywords)
#     if max_similarity<0.5:
#         return []
#     print(max_similarity)
#     print(": is the maximum similarity")
#     return most_similar_chain

# # Example usage


# input_keywords = ['keyword1', 'keyword4', 'keyword5', 'keyword7', 'keyword8', 'keyword9']

# most_similar_chain = most_similar_chain(input_keywords)
# print("Most similar chain:", most_similar_chain)


Date based sorting function

In [6]:
# def chain_item_sorting(list_of_lists):
#     # Sort the list of lists based on the date in each inner list
#     sorted_list = sorted(list_of_lists[1:], key=lambda x: datetime.strptime(x['date'], "%B %d, %Y"))

#     # Combine the sorted list with the first item (keywords)
#     result = [list_of_lists[0]] + sorted_list

#     return result

# # Example usage
# list_of_lists = [
#     ['keyword1', 'keyword2', 'keyword3'],
#     ['January 1, 2022', 'Text 1'],
#     ['January 3, 2022', 'Text 2'],
#     ['December 18, 2021', 'Text 3']
# ]

# sorted_list_of_lists = chain_item_sorting(list_of_lists)
# print(sorted_list_of_lists)


In [7]:
sample_text = "hello world keyword1 keyword2 keyword3"
sample_date = "January 3, 2020"
add_to_global_var(sample_text,sample_date)

C:\Users\Kartik Gounder\AppData\Local\Temp\ipykernel_6444\955989383.py:27: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  keyword_similarity = sum(nlp(keyword).similarity(nlp(input_keyword)) for keyword in keywords for input_keyword in input_keywords)


NameError: name 'text_date_pair' is not defined

In [ ]:
sample_text1 = "hello sample world keyword1 keyword2 keyword3"
sample_date1 = "April 10, 2024"
add_to_global_var(sample_text1,sample_date1)

In [ ]:
print_global_list()